In [36]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [37]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [38]:
folder = gen_config['folder']

path_prod_data = gen_config['path_prod_data']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [39]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [40]:
prod_data = pd.read_parquet(f"{folder}/{path_prod_data}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [41]:
df_pred_test = df_test[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test: {perdida}")
print(f"Primeras filas de test:\n{df_pred_test.iloc[:11,:4]}")

Perdida en test: nan
Primeras filas de test:
     product_id  tn_futuro_real  tn_prediccion_real        error
0         20001     1504.712094         1122.402907   -25.407464
490       20660        0.228250            2.814980  1133.288119
516       20697        2.598690            2.524008    -2.873813
517       20699        3.033950            1.095789   -63.882445
518       20700        3.113530            1.903591   -38.860685
519       20701        3.762145            2.615671   -30.473958
520       20702        2.030630            2.001942    -1.412774
521       20703        9.465700           11.038816    16.619119
522       20705        1.031987            1.597565    54.804727
523       20706        4.852250            2.601639   -46.382842
524       20708        4.317608            2.221622   -48.545069


In [42]:
# df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
# df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
# df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

# df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

In [43]:
df_pred_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
#df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
#df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
#df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
#df_pred_fut.sort_values('weight', ascending=False, inplace=True)

print(f"Primeras filas de futuro:\n{df_pred_fut.iloc[:11,:4]}")

Primeras filas de futuro:
    product_id  tn_futuro_real  tn_prediccion_real
0        20001             0.0         1011.286665
1        20002             0.0          913.046864
2        20003             0.0          633.758468
3        20004             0.0          446.395986
4        20005             0.0          415.371962
5        20006             0.0          381.599315
6        20007             0.0          392.840248
7        20008             0.0          310.953633
8        20009             0.0          360.457605
9        20010             0.0          263.229889
10       20011             0.0          290.947914


In [44]:
# df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
# df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
# df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [45]:
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')

In [46]:
kaggle =  df_pred_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
kaggle = kaggle.merge(prod_a_predecir, on='product_id', how='right')
kaggle['tn'] = kaggle['tn'].fillna(0)

In [51]:
total = kaggle['tn'].sum()
correccion = total/30600
kaggle['tn'] = kaggle['tn']/correccion
print(total)

20970.58146783708


In [52]:
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)

In [49]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

---------------------------------------06_evaluar_prediccion----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



